In [1]:
/****************************
* Interview
****************************/

* Import data 
import excel "/accounts/gen/guest/ppori34/interview/raw/Sampledata_RA.xlsx", sheet("Sheet1") firstrow clear

* Rename the column names
rename wsdateofincorporation date_of_incorporation
rename wsprimarysiccode primary_sic_code
rename wsprivateindicator private_indicator
rename wssdccusip sdc_cusip
rename wssedol sedol
rename wsticker ticker
rename wscid company_id
rename wscash cash
rename wscashandstinvestments cash_and_short_term_investments
rename wscommonsharesoutstanding common_shares_outstanding
rename wscommonstock common_stock
rename wscostofgoodssold cost_of_goods_sold
rename wsearningsbeforeinttaxesanddepr ebitd
rename wsexports exports
rename wsnetincome net_income
rename wsreturnonassets return_on_assets
rename wssales sales
rename wstotalassets total_assets
rename wscurrentassets current_assets
rename wscurrentliabilities current_liabilities
rename wstotalltdebt total_liabilities_and_debt
rename wsppe property_plant_equipment
rename wscashflow cash_flow
rename wscurrentmarketcap current_market_cap
rename wsforeignloans foreign_loans
rename wsinternationalassets international_assets
rename wsinternationaloperatingincome international_operating_income
rename wsinternationalsales international_sales
rename wssalesgaap sales_gaap
rename wssalesusd sales_usd


(34 vars, 550 obs)
































In [2]:
* Check missing values with 550 observations
summarize property_plant_equipment current_assets current_liabilities cash ebitd sales sales_usd employees


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
property_p~t |        449    29416.76    74844.39          0     506468
current_as~s |        392    74294.35    208830.9    2.26196    1400000
current_li~s |        392    58084.81    199823.4    .043565    1600000
        cash |        310    17817.63    42817.84       .032     289169
       ebitd |        418    8137.829     22272.5     -51733     163020
-------------+---------------------------------------------------------
       sales |        461    106651.2      296112    .268026    1900000
   sales_usd |        461    1461.297    2932.917     .11525    17151.4
   employees |        406    5889.204    10228.21          0      80590


In [3]:
* Handle missing values
* Replace missing values in key variables with group-level median to maintain panel structure
foreach var of varlist property_plant_equipment current_assets current_liabilities cash ebitd sales sales_usd employees {
    bysort country year: egen median_`var' = median(`var')
    replace `var' = median_`var' if missing(`var')
    drop median_`var'
}


(10 missing values generated)
(91 real changes made)
(10 missing values generated)
(148 real changes made)
(10 missing values generated)
(148 real changes made)
(10 missing values generated)
(230 real changes made)
(10 missing values generated)
(122 real changes made)
(10 missing values generated)
(79 real changes made)
(10 missing values generated)
(79 real changes made)
(10 missing values generated)
variable employees was long now double
(134 real changes made)


In [4]:
/****************************
- Earning before taxes and interest includes depreciation
- Operating Income = Earnings before interest and taxes = ebitd(Earnings before interest,taxes, and depreciaton
****************************/

* 1> Create the following new variables:

* ROIC (Return on Invested Capital) = Operating Income / Invested Capital at time t-1
* groupping by country year 
gen invested_capital = property_plant_equipment + current_assets - current_liabilities - cash
bysort country iso (year): gen roic = ebitd / invested_capital[_n-1] if invested_capital[_n-1] != .

* b. Annual Growth in Sales
bysort country iso (year): gen growth_sales = (sales - sales[_n-1]) / sales[_n-1] if sales[_n-1] != .

* Age variables still missing due to the unknown date of incorporation for missing ones
* Age of the firm
gen year_of_incorporation = year(date_of_incorporation)
gen age = year - year_of_incorporation

* Labor Productivity
gen labor_productivity = sales_usd / employees


(10 missing values generated)

(15 missing values generated)

(15 missing values generated)

(396 missing values generated)

(396 missing values generated)

(12 missing values generated)


In [5]:
sum invested_capital growth_sales age labor_productivity


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
invested_c~l |        540    25901.09    74179.99     -37681     524065
growth_sales |        535    58.32899    565.5141  -.9999464   11180.22
         age |        154    13.64286    17.45383         -7         75
labor_prod~y |        538    .3508946    .6547335   .0000254    8.22107


In [6]:
*>2 Provide the following summary statistics on each of the variables created above as well as the input variables

* Summary statistics 
foreach var of varlist invested_capital roic property_plant_equipment current_assets current_liabilities cash{
      summarize `var', detail
}



                      invested_capital
-------------------------------------------------------------
      Percentiles      Smallest
 1%       -16470         -37681
 5%     -439.559         -19735
10%     -97.4735         -18910       Obs                 540
25%     58.40325         -17782       Sum of wgt.         540

50%       560.96                      Mean           25901.09
                        Largest       Std. dev.      74179.99
75%     9511.155         464476
90%       102759         471591       Variance       5.50e+09
95%     153635.5       515357.5       Skewness       4.019953
99%       407364         524065       Kurtosis       21.23459

                            roic
-------------------------------------------------------------
      Percentiles      Smallest
 1%    -19.16208      -346.2826
 5%    -4.184739      -236.9447
10%    -.5429792       -63.9446       Obs                 535
25%     .0116372      -40.22919       Sum of wgt.         535

50%      .169185 

In [7]:
foreach var of varlist sales growth_sales labor_productivity sales_usd employees{
        summarize `var', detail
}



                           wssales
-------------------------------------------------------------
      Percentiles      Smallest
 1%      1.05683        .268026
 5%     2.955475        .575716
10%      29.8216           .977       Obs                 540
25%      264.735          1.003       Sum of wgt.         540

50%      2298.42                      Mean           93089.91
                        Largest       Std. dev.      275549.5
75%      28707.8        1700000
90%       218518        1700000       Variance       7.59e+10
95%       666320        1800000       Skewness       4.445166
99%      1700000        1900000       Kurtosis       24.20127

                        growth_sales
-------------------------------------------------------------
      Percentiles      Smallest
 1%    -.9988888      -.9999464
 5%    -.9916445      -.9998994
10%    -.9691262      -.9998898       Obs                 535
25%    -.7887139      -.9992823       Sum of wgt.         535

50%            0 

In [8]:
foreach var of varlist age year year_of_incorporation {
    summarize `var', detail
}



                             age
-------------------------------------------------------------
      Percentiles      Smallest
 1%           -7             -7
 5%           -4             -7
10%            0             -6       Obs                 154
25%            5             -6       Sum of wgt.         154

50%           10                      Mean           13.64286
                        Largest       Std. dev.      17.45383
75%           14             72
90%           29             73       Variance       304.6363
95%           68             74       Skewness       2.342443
99%           74             75       Kurtosis        8.19664

                            year
-------------------------------------------------------------
      Percentiles      Smallest
 1%         2000           2000
 5%         2000           2000
10%         2001           2000       Obs                 550
25%         2002           2000       Sum of wgt.         550

50%         2005       

In [9]:
* Missing varables
misstable summarize

                                                               Obs<.
                                                +------------------------------
               |                                | Unique
      Variable |     Obs=.     Obs>.     Obs<.  | values        Min         Max
  -------------+--------------------------------+------------------------------
  date_of_in~n |       396                 154  |     14      -8791       17478
  primary_si~e |       110                 440  |     38        133        8999
  private_in~r |       550                   0  |      0          .           .
          cash |        10                 540  |    356       .032      289169
  cash_and_s~s |       115                 435  |    435       .002      301303
  common_sha~g |        97                 453  |    241        .05     5535.01
  common_stock |       142                 408  |    146    .031273      112448
  cost_of_go~d |       144                 406  |    400    .471411     17

In [10]:
* 3> Winsorize the top and bottom 1% of the values for ROIC and Labor Productivity
bysort country year: egen roic_p1 = pctile(roic), p(1)
bysort country year: egen roic_p99 = pctile(roic), p(99)

replace roic = max(roic, roic_p1) if roic < roic_p1
replace roic = min(roic, roic_p99) if roic > roic_p99

gen roic_w = roic

bysort country year: egen labor_productivity_p1 = pctile(labor_productivity), p(1)
bysort country year: egen labor_productivity_p99 = pctile(labor_productivity), p(99)

replace labor_productivity = max(labor_productivity, labor_productivity_p1) if labor_productivity < labor_productivity_p1
replace labor_productivity = min(labor_productivity, labor_productivity_p99) if labor_productivity > labor_productivity_p99

gen labor_productivity_w = labor_productivity


(10 missing values generated)

(10 missing values generated)

(0 real changes made)

(5 real changes made)

(10 missing values generated)

(10 missing values generated)

(10 missing values generated)

(0 real changes made)

(2 real changes made)

(10 missing values generated)


In [11]:
summarize roic_w labor_productivity_w, detail


                           roic_w
-------------------------------------------------------------
      Percentiles      Smallest
 1%    -19.16208      -346.2826
 5%    -3.885424      -236.9447
10%    -.5242378       -63.9446       Obs                 540
25%     .0116984      -40.22919       Sum of wgt.         540

50%      .169185                      Mean           3.420015
                        Largest       Std. dev.      34.54261
75%     .5717771        203.504
90%     2.709135         255.96       Variance       1193.192
95%      8.96098       292.0069       Skewness       2.187658
99%      156.191       334.5901       Kurtosis       60.35271

                    labor_productivity_w
-------------------------------------------------------------
      Percentiles      Smallest
 1%     .0000913       .0000254
 5%      .007183       .0000498
10%     .0294795       .0000519       Obs                 540
25%     .0695424       .0000635       Sum of wgt.         540

50%       .1734

In [12]:
* 4> Compute median ROIC for each country each year
bysort country year: egen median_roic = median(roic_w)

save "/accounts/gen/guest/ppori34/interview/raw/Updated_Sampledata_RA.dta",replace


(10 missing values generated)

(file /accounts/gen/guest/ppori34/interview/raw/Updated_Sampledata_RA.dta not
    found)
file /accounts/gen/guest/ppori34/interview/raw/Updated_Sampledata_RA.dta saved


In [13]:
* 5> Merge the dataset with the country-year-level variables in the data “WEO_Data”. 
import excel "/accounts/gen/guest/ppori34/interview/raw/WEO_Data.xlsx", firstrow clear
keep iso Country SubjectDescriptor Units Q-AA

* Rename columns to reflect actual years with a prefix
rename Country country
rename Q year_2000
rename R year_2001
rename S year_2002
rename T year_2003
rename U year_2004
rename V year_2005
rename W year_2006
rename X year_2007
rename Y year_2008
rename Z year_2009
rename AA year_2010

describe


(42 vars, 29 obs)















Contains data
 Observations:            29                  
    Variables:            15                  
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
iso             str72   %72s                  iso
country         str13   %13s                  Country
SubjectDescri~r str39   %39s                  Subject Descriptor
Units           str14   %14s                  Units
year_2000       double  %10.0g                2000
year_2001       double  %10.0g                2001
year_2002       double  %10.0g                2002
year_2003       double  %10.0g                2003
year_2004       double  %10.0g                2004
year_2005       double  %10.0g                2005
year_2006       double  %10.0g                2006
year_2007       

In [14]:
* Reshape the WEO data to long format
reshape long year_, i(iso country SubjectDescriptor Units) j(year)

* Rename the reshaped value column for clarity
rename year_ value

* Remove rows with missing year values
drop if missing(year)

* Step 2: Save the imported data as a Stata (.dta) file
save "/accounts/gen/guest/ppori34/interview/raw/Updated_WEO_DATA.dta", replace


(j = 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010)

Data                               Wide   ->   Long
-----------------------------------------------------------------------------
Number of observations               29   ->   319         
Number of variables                  15   ->   6           
j variable (11 values)                    ->   year
xij variables:
      year_2000 year_2001 ... year_2010   ->   year_
-----------------------------------------------------------------------------


(0 observations deleted)

(file /accounts/gen/guest/ppori34/interview/raw/Updated_WEO_DATA.dta not
    found)
file /accounts/gen/guest/ppori34/interview/raw/Updated_WEO_DATA.dta saved


In [15]:
use "/accounts/gen/guest/ppori34/interview/raw/Updated_WEO_DATA.dta", clear
duplicates report country year

collapse (mean) value, by(country year)

save "/accounts/gen/guest/ppori34/interview/raw/Updated_WEO_DATA_collapsed.dta", replace

* Now merge the collapsed dataset
use "/accounts/gen/guest/ppori34/interview/raw/Updated_Sampledata_RA.dta", clear
merge m:1 country year using "/accounts/gen/guest/ppori34/interview/raw/Updated_WEO_DATA_collapsed.dta"

save "/accounts/gen/guest/ppori34/interview/raw/merged.dta", replace




Duplicates in terms of country year

--------------------------------------
   Copies | Observations       Surplus
----------+---------------------------
        2 |           22            11
        3 |          297           198
--------------------------------------


(file /accounts/gen/guest/ppori34/interview/raw/Updated_WEO_DATA_collapsed.dta
    not found)
file /accounts/gen/guest/ppori34/interview/raw/Updated_WEO_DATA_collapsed.dta
    saved



    Result                      Number of obs
    -----------------------------------------
    Not matched                           176
        from master                       110  (_merge==1)
        from using                         66  (_merge==2)

    Matched                               440  (_merge==3)
    -----------------------------------------

(file /accounts/gen/guest/ppori34/interview/raw/merged.dta not found)
file /accounts/gen/guest/ppori34/interview/raw/merged.dta saved


In [16]:
* 6> For each country, generate a graph where one line shows the median Return on InvestedCapital (units on left y-axis) and a second line showing real GDP growth (units on right y-axis), both over time.

* Mergin with the subset of datasets
use "/accounts/gen/guest/ppori34/interview/raw/Updated_Sampledata_RA.dta", clear
keep country year median_roic
duplicates drop
save "/accounts/gen/guest/ppori34/interview/raw/median_roic_data.dta", replace

* Load the real GDP growth data from the WEO dataset
use "/accounts/gen/guest/ppori34/interview/raw/Updated_WEO_DATA.dta", clear

* Keep only the real GDP growth for merging
keep if SubjectDescriptor == "Gross domestic product, constant prices"
rename value gdp_growth
keep country year gdp_growth

* Save for merging
save "/accounts/gen/guest/ppori34/interview/raw/gdp_growth_data.dta", replace

* Load the median ROIC data and merge with GDP growth data
use "/accounts/gen/guest/ppori34/interview/raw/median_roic_data.dta", clear
merge m:1 country year using "/accounts/gen/guest/ppori34/interview/raw/gdp_growth_data.dta"

* Drop unmatched observations
keep if _merge == 3
drop _merge





Duplicates in terms of all variables

(495 observations deleted)

(file /accounts/gen/guest/ppori34/interview/raw/median_roic_data.dta not
    found)
file /accounts/gen/guest/ppori34/interview/raw/median_roic_data.dta saved


(220 observations deleted)



(file /accounts/gen/guest/ppori34/interview/raw/gdp_growth_data.dta not found)
file /accounts/gen/guest/ppori34/interview/raw/gdp_growth_data.dta saved



    Result                      Number of obs
    -----------------------------------------
    Not matched                            66
        from master                        11  (_merge==1)
        from using                         55  (_merge==2)

    Matched                                44  (_merge==3)
    -----------------------------------------

(66 observations deleted)



In [17]:
* Generate graph for each country
levelsof country, local(countries)

foreach c of local countries {
    keep if country == "`c'"
    twoway ///
        (line median_roic year, yaxis(1) lcolor(blue) lpattern(solid) lwidth(medium)) ///
        (line gdp_growth year, yaxis(2) lcolor(red) lpattern(dash) lwidth(medium)), ///
        title("ROIC and GDP Growth for `c' Over Time") ///
        ytitle("Median ROIC (%)") ///
        ytitle("GDP Growth (%)", axis(2)) ///
        legend(order(1 "Median ROIC" 2 "GDP Growth")) ///
        xlabel(2000(1)2010)

    graph export "graph_`c'.png", replace

    use "/accounts/gen/guest/ppori34/interview/raw/median_roic_data.dta", clear
    merge m:1 country year using "/accounts/gen/guest/ppori34/interview/raw/gdp_growth_data.dta"
    keep if _merge == 3
    drop _merge
}


`"China"' `"Germany"' `"Japan"' `"South Africa"'

(33 observations deleted)
file /accounts/gen/guest/ppori34/.stata_kernel_cache/graph0.svg saved as SVG
    format
file /accounts/gen/guest/ppori34/.stata_kernel_cache/graph0.pdf saved as PDF
    format
(file graph_China.png not found)
file graph_China.png written in PNG format

    Result                      Number of obs
    -----------------------------------------
    Not matched                            66
        from master                        11  (_merge==1)
        from using                         55  (_merge==2)

    Matched                                44  (_merge==3)
    -----------------------------------------
(66 observations deleted)
(33 observations deleted)
file /accounts/gen/guest/ppori34/.stata_kernel_cache/graph1.svg saved as SVG
    format
file /accounts/gen/guest/ppori34/.stata_kernel_cache/graph1.pdf saved as PDF
    format
(file graph_Germany.png not found)
file graph_Germany.png written in PNG format
